In [1]:
import os
import re
import json
from bs4 import BeautifulSoup

In [2]:
input_dir = "/Users/mateuszbulanda-gorol/Desktop/Projects/rag_lex_project/data/raw_data"
output_file = "/Users/mateuszbulanda-gorol/Desktop/Projects/rag_lex_project/data/prepared_dataset.json"

In [3]:
def html_to_text(html_content:str) -> str:
    """
    Parses HTML into plain text, removes unnecessary spaces and new lines.
    """
    soup = BeautifulSoup(html_content, "html.parser")

    for tag in soup(["style", "script", "head", "title", "meta"]):
        tag.decompose()

    text = soup.get_text(separator="\n")
    text = re.sub(f"\s+", " ", text)
    text = text.strip()
    return text

<>:11: SyntaxWarning: invalid escape sequence '\s'
<>:11: SyntaxWarning: invalid escape sequence '\s'
/var/folders/ms/zrsm9ljx6bjds2r5n__t93th0000gn/T/ipykernel_68775/714111337.py:11: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub(f"\s+", " ", text)


In [4]:
def chunk_text(text:str, max_chars:int=1000, overlap:int=100):
    """
    Splits the text into blocks with a maximum length of `max_chars`, with overlap.
    """
    chunks = []
    start = 0
    while start < len(text):
        end = start + max_chars
        chunk = text[start:end]
        chunks.append(chunk.strip())
        start = end - overlap
    return chunks

In [5]:
def process_html_files(input_dir:str, output_file:str, max_chars:int=1000):
    """
    Converts all HTML files in the directory into a list of chunks with metadata.
    """
    dataset = []
    for fname in os.listdir(input_dir):
        if not fname.endswith(".html"):
            continue
        path = os.path.join(input_dir, fname)
        with open(path, "r", encoding="utf-8") as f:
            html = f.read()
        
        text = html_to_text(html)
        chunks = chunk_text(text, max_chars=max_chars)
        parts = fname.replace(".html", "").split("_")
        metadata = {}

        if len(parts) >= 3:
            metadata = {
                "publisher": parts[0]
                ,"year": parts[1]
                ,"pos": parts[2]
                ,"filename": fname
            }
        
        for i, chunk in enumerate(chunks):
            dataset.append({
                "text": chunk
                ,"metadata": {**metadata, "chunk_id": i}
            })

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(dataset, f, ensure_ascii=False, indent=2)
    
    print(f"{len(dataset)} Chunks saved to {output_file}")
    return dataset

In [6]:
dataset = process_html_files(input_dir, output_file, max_chars=1000)
print("Example: ", dataset[0])

92948 Chunks saved to /Users/mateuszbulanda-gorol/Desktop/Projects/rag_lex_project/data/prepared_dataset.json
Example:  {'text': 'Rozporządzenie Ministra Finansów, Funduszy i Polityki Regionalnej z dnia 17 grudnia 2020 r. zmieniające rozporządzenie w sprawie udzielania pomocy na działalność badawczo-rozwojową związaną z wystąpieniem pandemii COVID-19, pomocy inwestycyjnej na infrastrukturę służącą do testowania i przygotowania do masowej produkcji produktów służących zwalczaniu pandemii COVID-19 oraz pomocy inwestycyjnej na produkcję produktów służących zwalczaniu pandemii COVID-19, w ramach programów operacyjnych na lata 2014-2020 1) Minister Finansów, Funduszy i Polityki Regionalnej kieruje działem administracji rządowej - rozwój regionalny, na podstawie § 1 ust. 2 pkt 4 rozporządzenia Prezesa Rady Ministrów z dnia 6 października 2020 r. w sprawie szczegółowego zakresu działania Ministra Finansów, Funduszy i Polityki Regionalnej (Dz. U. poz. 1719). Treść rozporządzenia Na podstawie a